In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Milestone - 4

In [ ]:
#----------------------------- DON'T CHANGE THIS --------------------------
DATA_SEED = 67
TRAINING_SEED = 1234
MAX_LEN = 50
BATCH_SIZE = 32
EMB_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 5

In [ ]:
import random
import torch
torch.backends.cudnn.enabled = False
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
random.seed(DATA_SEED)
np.random.seed(DATA_SEED)
torch.manual_seed(DATA_SEED)
torch.cuda.manual_seed(DATA_SEED)

In [ ]:
data_path= '../Data/train.csv'     # enter your data path here
df = pd.read_csv(data_path)       # read it and store it in df

In [ ]:
# Split train df into train_df(80%) and test_df (20%) use seed
# ------------------- write your code here -------------------------------
#-------------------------------------------------------------------------
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    random_state=DATA_SEED
)

In [ ]:
# create a simple space-based tokenizer.
# ------------------- write your code here -------------------------------
#-------------------------------------------------------------------------
def tokenize(text):
    """A simple space-based tokenizer."""
    return str(text).lower().split(' ')

In [ ]:
from collections import Counter

In [ ]:
# Use counter to count all tokens in train_df
token_counter = Counter()
# ------------------- write your code here -------------------------------
#------------------------------------------------------------------------
for text in train_df['text']:
    tokens = tokenize(text)
    token_counter.update(tokens)
#------------------------------------------------------------------------
print("Vocabulary build complete.")

In [ ]:
#----------------------------- DON'T CHANGE THIS --------------------------
specials = ['<unk>', '<pad>']
min_freq = 2
vocab_list = specials + [token for token, freq in token_counter.items() if freq >= min_freq]
word2idx = {token: i for i, token in enumerate(vocab_list)}
# --- CRUCIAL ADDITIONS to make the provided code work ---
# The functions below depend on these values
UNK_IDX = word2idx['<unk>']
PAD_IDX = word2idx['<pad>']

In [ ]:
def text_pipeline(text):
    """Converts text to a list of indices using the word2idx dict."""
    tokens = tokenize(text)
    return [word2idx.get(token, UNK_IDX) for token in tokens]

In [ ]:
class EmotionDataset(Dataset):
    def __init__(self, dataframe):
        self.texts = dataframe['text'].values
        self.labels = dataframe[['anger', 'fear', 'joy', 'sadness', 'surprise']].values.astype(np.float32)
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]
def collate_batch(batch):
    label_list, text_list = [], []
    for (_text, _labels) in batch:
        label_list.append(_labels)
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)[:MAX_LEN]
        text_list.append(processed_text)
    label_list = torch.tensor(label_list, dtype=torch.float32)
    text_list = pad_sequence(text_list, batch_first=True, padding_value=PAD_IDX)
    if text_list.shape[1] < MAX_LEN:
        pad_tensor = torch.full(
            (text_list.shape[0], MAX_LEN - text_list.shape[1]),
            PAD_IDX,
            dtype=torch.int64
        )
        text_list = torch.cat((text_list, pad_tensor), dim=1)

    return text_list, label_list


# Create train and val dataloaders
# ------------------- write your code here -------------------------------
#------------------------------------------------------------------------
train_dataset = EmotionDataset(train_df)
val_dataset = EmotionDataset(val_df)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_batch
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False, # No need to shuffle validation data
    collate_fn=collate_batch
)
#------------------------------------------------------------------------
print("DataLoaders created successfully.")

In [ ]:
# Get vocabulary info
vocab_size = len(vocab_list)
print(f"Vocabulary size: {vocab_size}")
print(f"Padding token index (PAD_IDX): {PAD_IDX}")
print(f"Unknown token index (UNK_IDX): {UNK_IDX}")


In [ ]:
words_to_check = ['happy', 'alone', 'sad']
for word in words_to_check:
    idx = word2idx.get(word, UNK_IDX)
    print(f"Index of '{word}': {idx}")


In [ ]:
# Define an Embedding layer
embedding = torch.nn.Embedding(num_embeddings=vocab_size, embedding_dim=EMB_DIM, padding_idx=PAD_IDX)

# Take one batch from the train_dataloader
sample_batch = next(iter(train_dataloader))
text_batch, label_batch = sample_batch

print(f"\nInput batch shape: {text_batch.shape}")  # should be (BATCH_SIZE, MAX_LEN)

# Pass through embedding layer
embedded_output = embedding(text_batch)

print(f"Output shape of Embedding layer: {embedded_output.shape}")  # should be (BATCH_SIZE, MAX_LEN, EMB_DIM)


In [ ]:
lstm = torch.nn.LSTM(
    input_size=EMB_DIM,
    hidden_size=HIDDEN_DIM,
    num_layers=1,
    batch_first=True
)

In [ ]:
text_batch, label_batch = next(iter(train_dataloader))
print(f"Input batch shape: {text_batch.shape}")  # (BATCH_SIZE, MAX_LEN)


In [ ]:
embedded = embedding(text_batch)
print(f"After Embedding layer: {embedded.shape}")  # (BATCH_SIZE, MAX_LEN, EMB_DIM)

# Step 5: Pass through LSTM
output, (h_n, c_n) = lstm(embedded)

# Step 6: Print all shapes
print(f"LSTM output shape: {output.shape}")        # (BATCH_SIZE, MAX_LEN, HIDDEN_DIM)
print(f"Hidden state shape (h_n): {h_n.shape}")    # (num_layers, BATCH_SIZE, HIDDEN_DIM)
print(f"Cell state shape (c_n): {c_n.shape}")      # (num_layers, BATCH_SIZE, HIDDEN_DIM)")

In [ ]:
print(f"Hidden state shape (h_n): {h_n.shape}")

In [ ]:
gru = nn.GRU(
    input_size=EMB_DIM,
    hidden_size=HIDDEN_DIM,
    num_layers=1,
    batch_first=True
)


In [ ]:
x = torch.randn(BATCH_SIZE, MAX_LEN, EMB_DIM)
output, h_layer = gru(x)

In [ ]:
print("Output shape of GRU:", output.shape)
print("Hidden state shape of GRU:", h_layer.shape)

In [ ]:
bidir_lstm = nn.LSTM(
    input_size=EMB_DIM,
    hidden_size=HIDDEN_DIM,
    num_layers=1,
    batch_first=True,
    bidirectional=True
)

In [ ]:
x = torch.randn(BATCH_SIZE, MAX_LEN, EMB_DIM)

# forward pass
output, (h_n, c_n) = bidir_lstm(x)

In [ ]:
print("Output shape:", output.shape)
print("Hidden state shape:", h_n.shape)
print("Cell state shape:", c_n.shape)

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(5730, EMB_DIM)
        self.lstm = nn.LSTM(EMB_DIM, HIDDEN_DIM, batch_first=True)
        self.fc = nn.Linear(HIDDEN_DIM, OUTPUT_DIM)

    def forward(self, x):
        x = self.embedding(x)
        out, (h_n, c_n) = self.lstm(x)
        return self.fc(out[:, -1, :])  # take last hidden output

In [ ]:
lstm_model = LSTMModel()

In [ ]:
def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [ ]:
print(f"Total Trainable Parameters in LSTM Model: {count_params(lstm_model):,}")


In [ ]:
class BiLSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(5730, EMB_DIM)
        self.lstm = nn.LSTM(EMB_DIM, HIDDEN_DIM, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(HIDDEN_DIM * 2, OUTPUT_DIM)

    def forward(self, x):
        x = self.embedding(x)
        out, (h_n, c_n) = self.lstm(x)
        return self.fc(out[:, -1, :])

In [ ]:
bilstm_model = BiLSTM()

In [ ]:
print(f"Total Trainable Parameters in BILSTM Model: {count_params(bilstm_model):,}")

In [ ]:
class StackedGRU(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(5730, EMB_DIM)
        self.gru = nn.GRU(EMB_DIM, HIDDEN_DIM, num_layers=2, batch_first=True)
        self.fc = nn.Linear(HIDDEN_DIM, OUTPUT_DIM)

    def forward(self, x):
        x = self.embedding(x)
        out, h_n = self.gru(x)
        return self.fc(out[:, -1, :])

In [ ]:
stacked_GRU_model = StackedGRU()

In [ ]:
print(f"Total Trainable Parameters in Stacked GRU Model: {count_params(stacked_GRU_model):,}")

# Training the models

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score
import numpy as np
import wandb

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f" Using device: {device}")

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, output_dim, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(emb_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        out, (h_n, c_n) = self.lstm(x)
        return self.fc(out[:, -1, :])

In [ ]:
class GRUModel(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, output_dim, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_idx)
        self.gru = nn.GRU(emb_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        out, h_n = self.gru(x)
        return self.fc(out[:, -1, :])

In [ ]:
def train_and_evaluate(model_name,model, train_dataloader, val_dataloader, epochs=5, lr=1e-3):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    print(f"🚀Starting training of {model_name}🚀")

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0

        print(f"\n🟩 Starting Epoch {epoch+1}/{epochs}...")

        for batch_idx, (text, labels) in enumerate(train_dataloader):
            text, labels = text.to(device), labels.to(device)
            optimizer.zero_grad()

            # 🕵️ Debug prints (first batch only)
            if batch_idx == 0:
                print("Text shape:", text.shape)
                print("Label shape:", labels.shape)
                print("Text dtype:", text.dtype)
                print("Label dtype:", labels.dtype)

            outputs = model(text)

            if batch_idx == 0:
                print("Output shape:", outputs.shape)
                print("Output dtype:", outputs.dtype)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_dataloader)

        # ---------- Validation ----------
        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for text, labels in val_dataloader:
                text, labels = text.to(device), labels.to(device)
                outputs = model(text)
                preds = torch.sigmoid(outputs).cpu().numpy()
                all_preds.append(preds)
                all_labels.append(labels.cpu().numpy())

        all_preds = np.concatenate(all_preds)
        all_labels = np.concatenate(all_labels)
        all_preds_bin = (all_preds >= 0.5).astype(int)
        macro_f1 = f1_score(all_labels, all_preds_bin, average='macro')

        print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f} | Macro F1: {macro_f1:.4f}")

    print("✅ Training complete!")
    return model


In [ ]:
model_lstm = LSTMModel(len(vocab_list)+1, EMB_DIM, HIDDEN_DIM, OUTPUT_DIM, PAD_IDX).to(device)

In [ ]:
# trained_model = train_and_evaluate("LSTM Model",model_lstm, train_dataloader, val_dataloader, epochs=10)

In [ ]:
model_gru = GRUModel(len(vocab_list)+1, EMB_DIM, HIDDEN_DIM, OUTPUT_DIM, PAD_IDX).to(device)

In [ ]:
# trained_model_2 = train_and_evaluate("GRU Model",model_gru, train_dataloader, val_dataloader, epochs=10)

In [ ]:
class StackedGRU(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, output_dim, pad_idx, num_layers=5):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_idx)
        # num_layers=2 makes it a stacked GRU
        self.gru = nn.GRU(emb_dim, hidden_dim, batch_first=True, num_layers=num_layers)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        out, h_n = self.gru(x)
        # out[:, -1, :] -> last time step of the top layer
        return self.fc(out[:, -1, :])


In [ ]:
stacked_gru = StackedGRU(len(vocab_list)+1, EMB_DIM, HIDDEN_DIM, OUTPUT_DIM, PAD_IDX).to(device)

In [ ]:
trained_model_3 = train_and_evaluate("Stacked GRU Model",stacked_gru, train_dataloader, val_dataloader, epochs=50)

In [ ]:
import wandb
from sklearn.metrics import f1_score
import numpy as np
import torch

# Login to W&B using your API key
wandb.login(key="96513a18cdfc4585db6e8a169369fcc713e8ef2c")  # <-- yahan apna API key daal de

# Initialize W&B run
wandb.init(project="emotion_detection", name="Logged_Existing_Model")

# Putting our model in eval mode

# LSTM
# model_lstm.eval()
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model_lstm.to(device)



# GRU
# model_gru.eval()
# model_gru.to(device)


# Stacked_GRU
stacked_gru.eval()
stacked_gru.to(device)

all_preds, all_labels = [], []

# Going through validation data to collect predictions
with torch.no_grad():
    for text, labels in val_dataloader:
        text, labels = text.to(device), labels.to(device)
        outputs = stacked_gru(text)
        preds = torch.sigmoid(outputs).cpu().numpy()
        all_preds.append(preds)
        all_labels.append(labels.cpu().numpy())

all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

# Converting probabilities to binary predictions
all_preds_bin = (all_preds >= 0.5).astype(int)

# Computing macro F1 score
macro_f1 = f1_score(all_labels, all_preds_bin, average="macro")

# Logging to W&B
wandb.log({
    "val_macro_f1": macro_f1
})

print(f"✅ Logged Macro F1 to W&B: {macro_f1:.4f}")

wandb.finish()


In [ ]:
# test_df
test_df = pd.read_csv("../Data/test.csv")

In [ ]:
class TestDataset(Dataset):
    def __init__(self, dataframe):
        self.texts = dataframe['text'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]


In [ ]:
def collate_test_batch(batch):
    text_list = []
    for _text in batch:
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)[:MAX_LEN]
        text_list.append(processed_text)
    
    text_list = pad_sequence(text_list, batch_first=True, padding_value=PAD_IDX)
    if text_list.shape[1] < MAX_LEN:
        pad_tensor = torch.full(
            (text_list.shape[0], MAX_LEN - text_list.shape[1]),
            PAD_IDX,
            dtype=torch.int64
        )
        text_list = torch.cat((text_list, pad_tensor), dim=1)
    
    return text_list


In [ ]:
test_dataset = TestDataset(test_df)  # test_df me sirf text column hona chahiye
test_dataloader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_test_batch
)


In [ ]:
model = stacked_gru  # stacked GRU trained model
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

all_preds = []

with torch.no_grad():
    for text in test_dataloader:
        text = text.to(device)
        outputs = model(text)
        preds = torch.sigmoid(outputs).cpu().numpy()
        preds_bin = (preds >= 0.5).astype(int)
        all_preds.append(preds_bin)

final_preds = np.concatenate(all_preds)
print(final_preds.shape)  # (num_test_samples, 5)


In [ ]:
final_preds

In [ ]:
columns = ['anger', 'fear', 'joy', 'sadness', 'surprise']

# Create a DataFrame
df_preds = pd.DataFrame(final_preds, columns=columns)

# Optional: add an ID column
df_preds.insert(0, 'id', range(0, len(df_preds)))

In [ ]:
df_preds.to_csv("submission.csv", index=False)

print("✅ Predictions saved to test_predictions.csv")
